In [1]:
import paddle
import numpy as np
import os
import random

数据预处理`Load_data`

In [2]:
def load_data():
    # 从文件导入数据
    datafile = '../data/houseprice.data'
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]

    # 计算train数据集的最大值，最小值
    maximums, minimums = training_data.max(axis=0), training_data.min(axis=0)
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
   
    max_values = maximums
    min_values = minimums
    
    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - min_values[i]) / (maximums[i] - minimums[i])

    return training_data, test_data

In [3]:
training_data, test_data = load_data()
print(training_data.shape)
print(training_data[0,:])

(404, 14)
[0.         0.18       0.07344183 0.         0.3148148  0.57750523
 0.64160657 0.26920313 0.         0.2275574  0.28723404 1.
 0.0896799  0.42222223]


定义网络模型`class Model`

In [4]:
class Model(paddle.nn.Layer):

    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Model, self).__init__()
        
        # 定义一层全连接层，输入维度是13，输出维度是1
        self.fc = paddle.nn.Linear(in_features=13, out_features=1)
    
    # 网络的前向计算
    def forward(self, inputs):
        out = self.fc(inputs)
        return out


定义训练过程

In [5]:
# 声明定义好的线性回归模型
model = Model()
# 开启模型训练模式
model.train()
# 加载数据
training_data, test_data = load_data()
# 定义优化算法，使用随机梯度下降SGD
# 学习率设置为0.01
opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())


EPOCH_NUM = 10   # 设置外层循环次数
BATCH_SIZE = 10  # 设置batch大小

# 定义外层循环
for epoch_id in range(EPOCH_NUM):
    # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
    np.random.shuffle(training_data)
    
    # 将训练数据进行拆分，每个batch包含10条数据
    mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
    
    # 定义内层循环
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(mini_batch[:, :-1]) # 获得当前批次训练数据
        y = np.array(mini_batch[:, -1:]) # 获得当前批次训练标签（真实房价）
        # 将numpy数据转为飞桨动态图tensor的格式
        house_features = paddle.to_tensor(x)
        prices = paddle.to_tensor(y)
        
        # 前向计算
        predicts = model(house_features)
        
        # 计算损失
        loss = paddle.nn.functional.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id%20==0:
            print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
        
        # 反向传播，计算每层参数的梯度值
        avg_loss.backward()
        # 更新参数，根据设置好的学习率迭代一步
        opt.step()
        # 清空梯度变量，以备下一轮计算
        opt.clear_grad()

W0713 19:48:25.390617   655 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.0, Runtime API Version: 11.7
W0713 19:48:25.392081   655 gpu_resources.cc:91] device: 0, cuDNN Version: 8.9.


epoch: 0, iter: 0, loss is: [0.13924897]
epoch: 0, iter: 20, loss is: [0.03704375]
epoch: 0, iter: 40, loss is: [0.13377792]
epoch: 1, iter: 0, loss is: [0.09302473]
epoch: 1, iter: 20, loss is: [0.10062233]
epoch: 1, iter: 40, loss is: [0.10743545]
epoch: 2, iter: 0, loss is: [0.13910048]
epoch: 2, iter: 20, loss is: [0.01937735]
epoch: 2, iter: 40, loss is: [0.04345099]
epoch: 3, iter: 0, loss is: [0.07132774]
epoch: 3, iter: 20, loss is: [0.06045696]
epoch: 3, iter: 40, loss is: [0.00901344]
epoch: 4, iter: 0, loss is: [0.08987448]
epoch: 4, iter: 20, loss is: [0.03745076]
epoch: 4, iter: 40, loss is: [0.07416233]
epoch: 5, iter: 0, loss is: [0.03640366]
epoch: 5, iter: 20, loss is: [0.01735956]
epoch: 5, iter: 40, loss is: [0.04659626]
epoch: 6, iter: 0, loss is: [0.0315477]
epoch: 6, iter: 20, loss is: [0.02890182]
epoch: 6, iter: 40, loss is: [0.11034071]
epoch: 7, iter: 0, loss is: [0.09225357]
epoch: 7, iter: 20, loss is: [0.03790786]
epoch: 7, iter: 40, loss is: [0.28682932]
e

In [6]:
paddle.save(model.state_dict(), 'LR_model.pdparams')
print("模型保存成功，模型参数保存在LR_model.pdparams中")

模型保存成功，模型参数保存在LR_model.pdparams中


In [7]:
def load_one_example():
    # 从上边已加载的测试集中，随机选择一条作为测试数据
    idx = np.random.randint(0, test_data.shape[0])
    idx = -10
    one_data, label = test_data[idx, :-1], test_data[idx, -1]
    # 修改该条数据shape为[1,13]
    one_data =  one_data.reshape([1,-1])

    return one_data, label

In [8]:
# 参数为保存模型参数的文件地址
model_dict = paddle.load('LR_model.pdparams')
model.load_dict(model_dict)
model.eval()

# 参数为数据集的文件地址
one_data, label = load_one_example()
# 将数据转为动态图的variable格式 
one_data = paddle.to_tensor(one_data)
predict = model(one_data)

# 对结果做反归一化处理
predict = predict * (max_values[-1] - min_values[-1]) + min_values[-1]
# 对label数据做反归一化处理
label = label * (max_values[-1] - min_values[-1]) + min_values[-1]

print("Inference result is {}, the corresponding label is {}".format(predict.numpy(), label))

Inference result is [[24.107462]], the corresponding label is 19.700000762939453
